In [1]:
# If you haven't installed the 'pg8000' package for PostgreSQL connection, add it to your Lambda function
#!pip install pg8000

In [2]:
import boto3
import pymysql
import os
from dotenv import load_dotenv

#load_dotenv(dotenv_path)

#method to get values of environment varaibles from .env file 
#load_dotenv('.env')

# Environment variables for sensitive data
DB_HOST = "database-1.cnq4kwysqjdb.us-east-1.rds.amazonaws.com"
DB_PORT = 3306
DB_NAME = "mission_green"
DB_USER = ""
DB_PASSWORD = ""

#DB_TABLE = os.environ.get("DB_TABLE")  # Table to check for data
GLUE_WORKFLOW_NAME = "Mission_Green_ETL_WorkFlow"  # Glue workflow to trigger

# Function to check if a table contains any data
def check_table_has_data():
    # Connect to MySQL database
    conn = pymysql.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        db=DB_NAME
    )

    # Create a cursor to execute SQL statements
    cursor = conn.cursor()
    cursor.execute(f"SELECT COUNT(*) FROM mission_green_dw.air_water_fact")
    
    result = cursor.fetchone()
    return result[0] >= 0  # True if count = 0, otherwise False

# Start the glue workflow aftermissio
def process_handler():
   # Define the correct region
    region_name = "us-east-1"
    
    # Create a Glue client with the specified region
    glue = boto3.client("glue", region_name=region_name)
    try:
        if check_table_has_data():
            # Trigger the Glue workflow
            response = glue.start_workflow_run(Name=GLUE_WORKFLOW_NAME)

            # Output the workflow run ID to confirm it was triggered
            workflow_run_id = response["RunId"]
            print(f"Glue workflow '{GLUE_WORKFLOW_NAME}' started with Run ID: {workflow_run_id}")
        else:
            print("Table is not created, Glue workflow not triggered")

    except glue.exceptions.EntityNotFoundException:
        print(f"Error: Glue workflow '{GLUE_WORKFLOW_NAME}' not found.")

    except NoCredentialsError:
        print("Error: No AWS credentials found. Please configure your AWS credentials.")

    except PartialCredentialsError:
        print("Error: Incomplete AWS credentials. Please check your configuration.")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    try:
        if check_table_has_data():
            # If table contains data, trigger the Glue workflow
            response = glue.start_workflow_run(Name=GLUE_WORKFLOW_NAME)
            print("Glue workflow triggered")
        else:
            print("Table is not created, Glue workflow not triggered")

    except Exception as e:
        print(f"Error occurred: {e}")
        
process_handler()

Glue workflow 'Mission_Green_ETL_WorkFlow' started with Run ID: wr_3be61151e161f7a7afa776ec183bf8b57bbc3fd2b08f152972fb34a21a953695
Glue workflow triggered
